# 1.3 资产组合 (Mixing Assets)

## 概念介绍

资产组合理论是现代投资理论的核心，由哈里·马科维茨(Harry Markowitz)于1952年提出。该理论研究如何通过组合不同的资产来获得最优的风险回报特性。通过在不相关或低相关的资产间进行多元化投资，可以降低投资组合的整体风险，同时保持一定的收益水平。

### 投资组合收益与风险

投资组合的期望收益率是各资产期望收益率的加权平均：

$E(R_p) = \sum_{i=1}^{n} w_i \cdot E(R_i)$

投资组合的方差（风险的平方）计算涉及资产间的协方差：

$\sigma_p^2 = \sum_{i=1}^{n} \sum_{j=1}^{n} w_i \cdot w_j \cdot \sigma_{ij}$

其中：
- $E(R_p)$ 是投资组合的期望收益率
- $w_i$ 是第i个资产的权重
- $E(R_i)$ 是第i个资产的期望收益率
- $\sigma_p^2$ 是投资组合的方差
- $\sigma_{ij}$ 是资产i和资产j之间的协方差

对于两资产组合，风险计算可以简化为：

$\sigma_p = \sqrt{w_1^2 \cdot \sigma_1^2 + w_2^2 \cdot \sigma_2^2 + 2 \cdot w_1 \cdot w_2 \cdot \rho_{12} \cdot \sigma_1 \cdot \sigma_2}$

其中：
- $\sigma_p$ 是投资组合的标准差（风险）
- $\sigma_1$ 和 $\sigma_2$ 是两个资产的标准差
- $\rho_{12}$ 是两个资产之间的相关系数

## 1.1 安装必要的库

首先安装需要用到的Python库

In [ ]:
# 安装必要的库
!pip install pandas numpy matplotlib yfinance

## 1.2 从Github加载数据

In [ ]:
!wget -O AAPL_data.csv "https://github.com/Newzil-git/Financial-Risk-Management/blob/update-repo/FR%20Code/Part%201/AAPL_data.csv"
!wget -O "US_national_debt _data.csv" "https://github.com/Newzil-git/Financial-Risk-Management/blob/update-repo/FR%20Code/Part%201/US_national_debt%20_data.csv"
file_path_aapl = "AAPL_data.csv"
file_path_debt = "US_national_debt _data.csv"

## 1.3 导入必要的库

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

## 1.4 数据加载和预处理

In [ ]:
# 读取AAPL数据和US国债数据
try:
    aapl_data = pd.read_csv(file_path_aapl)
    debt_data = pd.read_csv(file_path_debt)
    print("数据加载成功")
except Exception as e:
    print(f"数据加载失败: {e}")
    # 如果从Google Drive加载失败，尝试从当前目录加载
    try:
        aapl_data = pd.read_csv('AAPL_data.csv')
        debt_data = pd.read_csv('US_national_debt _data.csv')
        print("从当前目录加载数据成功")
    except Exception as e:
        print(f"数据加载再次失败: {e}")

In [ ]:
# 数据预览
print("AAPL数据预览:")
display(aapl_data.head())
print("\nUS国债数据预览:")
display(debt_data.head())

In [ ]:
# 提取收盘价数据并计算对数收益率
try:
    aapl_data['Close'] = pd.to_numeric(aapl_data['Close'], errors='coerce')
    debt_data['Close'] = pd.to_numeric(debt_data['Close'], errors='coerce')
    
    # 检查数据是否正确转换
    print("AAPL收盘价数据类型:", type(aapl_data['Close'].iloc[0]))
    print("US国债收盘价数据类型:", type(debt_data['Close'].iloc[0]))
    
    # 数据统计
    print("\nAAPL收盘价统计:")
    display(aapl_data['Close'].describe())
    print("\nUS国债收盘价统计:")
    display(debt_data['Close'].describe())
except Exception as e:
    print(f"数据类型转换错误: {e}")

## 1.5 计算收益率和波动率

In [ ]:
# 计算每日收益率（对数收益率）
aapl_returns = np.log(aapl_data['Close'] / aapl_data['Close'].shift(1)).dropna()
debt_returns = np.log(debt_data['Close'] / debt_data['Close'].shift(1)).dropna()

# 收益率预览
print("AAPL日收益率:")
display(aapl_returns.head())
print("\nUS国债日收益率:")
display(debt_returns.head())

In [ ]:
# 计算平均收益率和波动率（标准差）
avg_aapl = aapl_returns.mean()
avg_debt = debt_returns.mean()
std_aapl = aapl_returns.std()
std_debt = debt_returns.std()

# 计算资产间的相关系数
correlation = aapl_returns.corr(debt_returns)

# 显示结果
print(f"AAPL平均日收益率: {avg_aapl:.6f}")
print(f"AAPL波动率: {std_aapl:.6f}")
print(f"US国债平均日收益率: {avg_debt:.6f}")
print(f"US国债波动率: {std_debt:.6f}")
print(f"AAPL与US国债相关系数: {correlation:.6f}")

# 年化数据（假设一年252个交易日）
annual_avg_aapl = avg_aapl * 252
annual_avg_debt = avg_debt * 252
annual_std_aapl = std_aapl * np.sqrt(252)
annual_std_debt = std_debt * np.sqrt(252)

print("\n年化结果:")
print(f"AAPL年化收益率: {annual_avg_aapl:.2%}")
print(f"AAPL年化波动率: {annual_std_aapl:.2%}")
print(f"US国债年化收益率: {annual_avg_debt:.2%}")
print(f"US国债年化波动率: {annual_std_debt:.2%}")

## 1.6 计算不同权重的组合收益率和波动率

In [ ]:
# 计算不同权重下的投资组合收益率和波动率
weights = np.linspace(0, 1, 100)  # 生成100个权重值，从0到1
portfolio_returns = []
portfolio_volatility = []

for w in weights:
    # 组合的预期回报（日收益率）
    portfolio_return = w * avg_aapl + (1 - w) * avg_debt
    portfolio_returns.append(portfolio_return * 252)  # 年化
    
    # 组合的波动率
    portfolio_vol = np.sqrt(
        w ** 2 * std_aapl ** 2 + 
        (1 - w) ** 2 * std_debt ** 2 + 
        2 * w * (1 - w) * correlation * std_aapl * std_debt
    )
    portfolio_volatility.append(portfolio_vol * np.sqrt(252))  # 年化

# 找到最小波动率投资组合
min_vol_idx = np.argmin(portfolio_volatility)
min_vol_return = portfolio_returns[min_vol_idx]
min_vol_volatility = portfolio_volatility[min_vol_idx]
min_vol_weight = weights[min_vol_idx]

print(f"最小波动率组合:")
print(f"AAPL权重: {min_vol_weight:.2f}, US国债权重: {1-min_vol_weight:.2f}")
print(f"年化预期收益率: {min_vol_return:.2%}")
print(f"年化波动率: {min_vol_volatility:.2%}")

## 1.7 结果可视化

In [ ]:
# 绘制风险-回报曲线（有效前沿）
plt.figure(figsize=(12, 8))

# 绘制投资组合曲线
plt.plot(portfolio_volatility, portfolio_returns, 'b-', linewidth=3, label="投资组合组合")

# 绘制单个资产点
plt.scatter(annual_std_aapl, annual_avg_aapl, color='blue', s=100, label="AAPL (股票)", marker='o', edgecolors='black')
plt.scatter(annual_std_debt, annual_avg_debt, color='red', s=100, label="US国债 (债券)", marker='x', edgecolors='black')

# 标记最小波动率投资组合点
plt.scatter(min_vol_volatility, min_vol_return, color='green', s=150, label=f"最小波动率组合 (AAPL: {min_vol_weight:.2f})", marker='*', edgecolors='black')

# 添加投资组合权重标签
for i in range(0, len(weights), 10):  # 每10个点标记一个
    plt.annotate(f"{weights[i]:.1f}", 
                (portfolio_volatility[i], portfolio_returns[i]),
                xytext=(5, 0), 
                textcoords='offset points',
                fontsize=8)

# 设置图表属性
plt.title('投资组合风险-收益特性', fontsize=16)
plt.xlabel('年化波动率 (%)', fontsize=12)
plt.ylabel('年化预期收益率 (%)', fontsize=12)
plt.legend(loc='best', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

# 标注资产相关系数
plt.annotate(f"相关系数: {correlation:.2f}", 
             xy=(0.05, 0.05), 
             xycoords='axes fraction',
             fontsize=12,
             bbox=dict(boxstyle="round,pad=0.3", fc="white", ec="gray", alpha=0.8))

# 显示图表
plt.tight_layout()
plt.show()

In [ ]:
# 绘制权重与组合特性的关系图
fig, axs = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# 预期收益率与权重关系
axs[0].plot(weights, portfolio_returns, 'r-', linewidth=2)
axs[0].set_ylabel('年化预期收益率 (%)', fontsize=12)
axs[0].set_title('权重与预期收益率的关系', fontsize=14)
axs[0].grid(True, linestyle='--', alpha=0.7)
axs[0].axvline(x=min_vol_weight, color='green', linestyle='--', alpha=0.7, label='最小波动率权重')
axs[0].legend()

# 波动率与权重关系
axs[1].plot(weights, portfolio_volatility, 'b-', linewidth=2)
axs[1].set_xlabel('AAPL权重', fontsize=12)
axs[1].set_ylabel('年化波动率 (%)', fontsize=12)
axs[1].set_title('权重与波动率的关系', fontsize=14)
axs[1].grid(True, linestyle='--', alpha=0.7)
axs[1].axvline(x=min_vol_weight, color='green', linestyle='--', alpha=0.7, label='最小波动率权重')
axs[1].legend()

# 显示图表
plt.tight_layout()
plt.show()

## 1.8 结果分析与解释

通过上述分析与可视化，我们可以得出以下结论：

1. **资产特性**：
   - AAPL (股票)：年化预期收益率为{annual_avg_aapl:.2%}，年化波动率为{annual_std_aapl:.2%}
   - US国债 (债券)：年化预期收益率为{annual_avg_debt:.2%}，年化波动率为{annual_std_debt:.2%}
   - 两种资产的相关系数为{correlation:.2f}，表示{'较弱的正相关' if correlation > 0 and correlation < 0.3 else '较强的正相关' if correlation >= 0.3 and correlation < 0.7 else '强正相关' if correlation >= 0.7 else '负相关' if correlation < 0 else '无相关'}

2. **投资组合多元化收益**：
   - 通过组合这两种资产，我们可以构建具有不同风险和收益特性的投资组合
   - 最小波动率投资组合：AAPL权重为{min_vol_weight:.2f}，US国债权重为{1-min_vol_weight:.2f}
   - 该组合的年化预期收益率为{min_vol_return:.2%}，年化波动率为{min_vol_volatility:.2%}
   - 这一组合的波动率低于任一单一资产的波动率，体现了多元化投资的风险分散效果

3. **有效前沿分析**：
   - 图表展示的曲线代表了所有可能的投资组合组合
   - 曲线上的每一点代表一个特定的权重分配
   - 在给定风险水平下，越靠上的点提供更高的预期收益率

4. **投资策略建议**：
   - 风险厌恶型投资者可能更倾向于接近最小波动率投资组合的权重配置
   - 风险偏好型投资者可能更倾向于增加AAPL的权重以追求更高的收益
   - 投资者应根据自身风险承受能力、投资目标和时间跨度来调整投资组合权重
   
5. **局限性**：
   - 此模型基于历史数据，未来的实际表现可能不同
   - 分析假设资产收益率服从正态分布，而实际市场可能存在偏斜和尾部风险
   - 相关系数在不同市场条件下可能会变化，特别是在极端市场事件期间